<a href="https://colab.research.google.com/github/jrfanfan/Android_temp_con/blob/main/notebooks/starter_bikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

In [183]:
bikes.head()

,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
0,1/1/2011,0.0,3,13,3.0,3.0,0.7957,0.8,1,1,0,0
1,1/1/2011,1.0,8,30,1.7,1.7,0.8272,0.8,1,1,0,0
2,1/1/2011,2.0,5,26,1.9,1.9,0.8157,1.1,1,1,0,0
3,1/1/2011,3.0,3,9,2.5,2.5,0.7831,0.8,1,1,0,0
4,1/1/2011,4.0,0,1,2.0,2.0,0.8075,1.1,1,1,0,0


In [184]:
# Add casual and registered bikes rent per day
bikes['bikes_rent'] = bikes['casual'] + bikes['registered']


In [185]:
bikes['dteday'] = pd.to_datetime(bikes['dteday'])
bikes['day'] = bikes['dteday'].dt.day
bikes['month'] = bikes['dteday'].dt.month
bikes['year'] = bikes['dteday'].dt.year



In [186]:
# months: october, november and december.
bikes[bikes['month'].isin([10, 11, 12])]


,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday,bikes_rent,day,month,year
6551,2011-10-01,0.0,24,103,16.7,16.7,0.5709,21.7,1,4,0,0,127,1,10,2011
6552,2011-10-01,1.0,11,45,16.0,16.0,0.6260,25.2,1,4,0,0,56,1,10,2011
6553,2011-10-01,2.0,21,45,15.0,15.0,0.6690,21.8,1,4,0,0,66,1,10,2011
6554,2011-10-01,3.0,8,17,14.2,14.2,0.6213,27.3,2,4,0,0,25,1,10,2011
6555,2011-10-01,4.0,2,6,13.9,13.9,0.5915,27.0,2,4,0,0,8,1,10,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112470,2023-10-31,19.0,241,594,10.2,10.2,0.4516,8.4,2,4,0,1,835,31,10,2023
112471,2023-10-31,20.0,171,450,8.9,7.5,0.5268,9.3,1,4,0,1,621,31,10,2023
112472,2023-10-31,21.0,176,402,7.7,6.4,0.5756,7.8,1,4,0,1,578,31,10,2023
112473,2023-10-31,22.0,106,257,8.0,8.0,0.5604,0.0,1,4,0,1,363,31,10,2023


In [193]:
# Total bikes rent for a day
bikes['bikes_rent_day'] = bikes.groupby(['day', 'month', 'year'])['bikes_rent'].transform('sum')
bikes



,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday,bikes_rent,day,month,year,bikes_rent_day
0,2011-01-01,0.0,3,13,3.0,3.0,0.7957,0.8,1,1,0,0,16,1,1,2011,959
1,2011-01-01,1.0,8,30,1.7,1.7,0.8272,0.8,1,1,0,0,38,1,1,2011,959
2,2011-01-01,2.0,5,26,1.9,1.9,0.8157,1.1,1,1,0,0,31,1,1,2011,959
3,2011-01-01,3.0,3,9,2.5,2.5,0.7831,0.8,1,1,0,0,12,1,1,2011,959
4,2011-01-01,4.0,0,1,2.0,2.0,0.8075,1.1,1,1,0,0,1,1,1,2011,959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112470,2023-10-31,19.0,241,594,10.2,10.2,0.4516,8.4,2,4,0,1,835,31,10,2023,14702
112471,2023-10-31,20.0,171,450,8.9,7.5,0.5268,9.3,1,4,0,1,621,31,10,2023,14702
112472,2023-10-31,21.0,176,402,7.7,6.4,0.5756,7.8,1,4,0,1,578,31,10,2023,14702
112473,2023-10-31,22.0,106,257,8.0,8.0,0.5604,0.0,1,4,0,1,363,31,10,2023,14702


In [195]:
# Create a new dataframe called X that contians the features we're going
# to use to make predictions
X = bikes.drop(columns=['dteday', 'casual', 'registered', 'bikes_rent'])
X

,hr,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday,day,month,year,bikes_rent_day
0,0.0,3.0,3.0,0.7957,0.8,1,1,0,0,1,1,2011,959
1,1.0,1.7,1.7,0.8272,0.8,1,1,0,0,1,1,2011,959
2,2.0,1.9,1.9,0.8157,1.1,1,1,0,0,1,1,2011,959
3,3.0,2.5,2.5,0.7831,0.8,1,1,0,0,1,1,2011,959
4,4.0,2.0,2.0,0.8075,1.1,1,1,0,0,1,1,2011,959
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112470,19.0,10.2,10.2,0.4516,8.4,2,4,0,1,31,10,2023,14702
112471,20.0,8.9,7.5,0.5268,9.3,1,4,0,1,31,10,2023,14702
112472,21.0,7.7,6.4,0.5756,7.8,1,4,0,1,31,10,2023,14702
112473,22.0,8.0,8.0,0.5604,0.0,1,4,0,1,31,10,2023,14702
